In [1]:
pip install genai

Note: you may need to restart the kernel to use updated packages.


In [2]:
import google.generativeai as genai

In [3]:
from google.generativeai import types

In [4]:
import requests

In [5]:
import os

In [6]:
OPENWEATHER_API_KEY = "XXXXXXXXXXXXXX"

In [37]:
def get_current_temperature(location):
    """Fetch real-time temperature for a given location using OpenWeatherMap API."""
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={OPENWEATHER_API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
   
    if response.status_code == 200:
        return data['main']['temp']
    else:
        raise Exception(f"Error fetching weather: {data.get('message', 'Unknown error')}")

In [40]:
weather_function = {
    "name": "get_current_temperature",
    "description": "Gets the current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city name, e.g. San Francisco"
            },
        },
        "required": ["location"],
    },
}

In [41]:
GEMINI_API_KEY= "XXXXXXXXXXXXXX"
OPENWEATHER_API_KEY= "XXXXXXXXXXXXXX"

In [45]:
tools = [types.Tool(function_declarations=[weather_function])]

In [46]:
genai.configure(api_key=GEMINI_API_KEY)

In [47]:
# ✅ Attach tools directly here (not inside GenerationConfig)
model = genai.GenerativeModel(
    "gemini-2.5-flash",
        tools=tools
)

In [48]:
prompt = "What is the temperature in Bangalore, Karnataka?"
response = model.generate_content(prompt)

E0000 00:00:1759058430.055012 3535062 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [54]:
print(response.candidates[0].content.parts[0])

function_call {
  name: "get_current_temperature"
  args {
    fields {
      key: "location"
      value {
        string_value: "Bangalore"
      }
    }
  }
}



In [61]:
for candidate in response.candidates:
    for part in candidate.content.parts:
        if getattr(part,"function_call",None):
            fc = part.function_call
            print(f"Function:${fc.name}")
            print(f"Args:${fc.args}")
            if fc.name == "get_current_temperature":
                location = fc.args.get("location")
                try:
                    temperature = get_current_temperature(location)
                    print(f"The current  temperature in {location} is {temperature}°C ")
                except Exception as e:
                    print(f"Error: {e}")
            else:
                # If Gemini doesn't call the function, just show its response
                print("No function call found. Gemini says:")
                print(response.text)
            

Function:$get_current_temperature
Args:$<proto.marshal.collections.maps.MapComposite object at 0x1066aa650>
The current  temperature in Bangalore is 27.23°C 
